In [1]:
import globals
import os
import sqlite3
from sqlite3 import Error

from pathlib import Path
from shutil import rmtree

In [11]:
sqlite_path = Path(globals.sqlite_doc_vec_dir)
if sqlite_path.exists():
    rmtree(sqlite_path)
sqlite_path.mkdir(parents=True)

conn = sqlite3.connect(os.path.join(globals.sqlite_doc_vec_dir,globals.sqlite_doc_vec_file))
try:
    sql = "create table vectors (docid nvarchar(512), vector text, primary key (docid))"
    conn.execute(sql)
except Error as e:
    print(e)


In [12]:
# load the vectors
count = 0
rows = []
c = conn.cursor()
with open(os.path.join(globals.avg_doc_vec_dir, globals.avg_doc_vec_file), encoding='utf-8') as fp: 
    while True: 
        line = fp.readline() 
        if not line: 
            break

        fields = line.split('\t')
        rows.append((fields[0], fields[1]))
        count += 1
        if count % 10000 == 0: 
            sql = 'insert into vectors (docid, vector) values (?,?)'
            c.executemany(sql, rows)
            conn.commit()
            print ("Inserted:", str(count))
            rows = []
if len(rows) > 0:
    sql = 'insert into vectors (docid, vector) values (?,?)'
    c.executemany(sql, rows)
    conn.commit()
    print ("Inserted:", str(count))


Inserted: 10000
Inserted: 20000
Inserted: 30000
Inserted: 40000
Inserted: 50000
Inserted: 57939
